In [ ]:
import os
import base64
import subprocess
import sys
from time import sleep

# Safely run pip install without noisy output
subprocess.run(
    [sys.executable, "-m", "pip", "install", "py3Dmol", "ipywidgets"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

# 📚 Import libraries
import py3Dmol
import requests
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# 📂 Directory where multiple CIF models are stored
model_dir = "protein_fold_model"
available_models = sorted([
    f for f in os.listdir(model_dir) if f.endswith(".cif")
])

# Mapping temp to filename
def get_model_path_from_temperature(temp: int) -> str:
    fname = f"temp_{temp}.cif"
    path = os.path.join(model_dir, fname)
    if not os.path.isfile(path):
        raise FileNotFoundError(f"No CIF file for temperature {temp} at {path}")
    return path

# ─────────────────────────────────────────────────────────────────────────────
# Create a full‐page animated background behind all widgets
# ─────────────────────────────────────────────────────────────────────────────

bg_html = widgets.HTML("""
<div id="fullpage-bg"></div>
<style>
  #fullpage-bg {
    position: fixed;
    top: 0;
    right: 0;
    bottom: 0;
    left: 0;
    z-index: -999;
    background: linear-gradient(
      135deg,
      #E6F2FF 0%,
      #F0F8FF 50%,
      #E6F9FF 100%
    );
    background-size: 400% 400%;
    animation: fullpageBG 15s ease infinite;
  }
  @keyframes fullpageBG {
    0%   { background-position: 0%   50%; }
    50%  { background-position: 100% 50%; }
    100% { background-position: 0%   50%; }
  }
</style>
""")
display(bg_html)

# 🎨 Enhanced UI Styling with Modern Design
custom_css = """
<style>
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700;800&family=JetBrains+Mono:wght@400;500&display=swap');

body {
  margin: 0;
  padding: 0;
  background: linear-gradient(
    135deg,
    #F0F8FF 0%,
    #E0F8FF 50%,
    #F8FFFF 100%
  );
  background-size: 400% 400%;
  animation: pulseBG 15s ease-in-out infinite;
}
@keyframes pulseBG {
  0% { background-position: 0% 50%; }
  50% { background-position: 100% 50%; }
  100% { background-position: 0% 50%; }
}

:root {
    --primary-blue: #2563eb;
    --primary-blue-dark: #1d4ed8;
    --secondary-purple: #7c3aed;
    --accent-cyan: #06b6d4;
    --success-green: #10b981;
    --warning-amber: #f59e0b;
    --neutral-50: #f9fafb;
    --neutral-100: #f3f4f6;
    --neutral-200: #e5e7eb;
    --neutral-300: #d1d5db;
    --neutral-600: #4b5563;
    --neutral-700: #374151;
    --neutral-800: #1f2937;
    --neutral-900: #111827;
    --glass-bg: rgba(255, 255, 255, 0.1);
    --glass-border: rgba(255, 255, 255, 0.2);
}
* { box-sizing: border-box; }

.polar-container {
    font-family: 'Inter', -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 50%, #667eea 100%);
    background-size: 400% 400%;
    animation: gradientShift 15s ease infinite;
    border-radius: 24px;
    padding: 3rem 2rem;
    box-shadow: 
        0 25px 50px -12px rgba(0, 0, 0, 0.25),
        0 0 0 1px rgba(255, 255, 255, 0.1);
    margin: 1rem 0 2rem 0;
    position: relative;
    overflow: hidden;
    backdrop-filter: blur(20px);
}
@keyframes gradientShift {
    0%, 100% { background-position: 0% 50%; }
    50% { background-position: 100% 50%; }
}
.polar-container::before {
    content: '';
    position: absolute;
    top: 0;
    left: 0;
    right: 0;
    bottom: 0;
    background: url('data:image/svg+xml,<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 100 100"><defs><pattern id="snowflakes" x="0" y="0" width="25" height="25" patternUnits="userSpaceOnUse"><circle cx="3" cy="3" r="1.2" fill="rgba(255,255,255,0.08)"/><circle cx="15" cy="10" r="0.8" fill="rgba(255,255,255,0.12)"/><circle cx="22" cy="18" r="1" fill="rgba(255,255,255,0.06)"/><circle cx="8" cy="20" r="0.6" fill="rgba(255,255,255,0.1)"/></pattern></defs><rect width="100" height="100" fill="url(%23snowflakes)"/></svg>') repeat;
    pointer-events: none;
    animation: snow 25s linear infinite;
    opacity: 0.7;
}
@keyframes snow {
    0% { transform: translateY(-100px) rotate(0deg); }
    100% { transform: translateY(100vh) rotate(360deg); }
}
.polar-container::after {
    content: '';
    position: absolute;
    top: -50%;
    left: -50%;
    width: 200%;
    height: 200%;
    background: radial-gradient(circle, rgba(255,255,255,0.1) 0%, transparent 70%);
    animation: aurora 20s linear infinite;
    pointer-events: none;
}
@keyframes aurora {
    0%, 100% { transform: rotate(0deg) scale(1); opacity: 0.3; }
    33% { transform: rotate(120deg) scale(1.1); opacity: 0.6; }
    66% { transform: rotate(240deg) scale(0.9); opacity: 0.4; }
}

.polar-title {
    color: white;
    font-size: 3rem;
    font-weight: 800;
    text-align: center;
    margin: 0 0 1rem 0;
    text-shadow: 
        0 4px 8px rgba(0,0,0,0.3),
        0 2px 4px rgba(0,0,0,0.2);
    position: relative;
    z-index: 1;
    letter-spacing: -0.02em;
}
.polar-subtitle {
    color: rgba(255,255,255,0.95);
    font-size: 1.25rem;
    text-align: center;
    margin: 0 0 2rem 0;
    font-weight: 400;
    position: relative;
    z-index: 1;
    letter-spacing: 0.01em;
}

.controls-panel {
    background: linear-gradient(135deg, 
        rgba(255,255,255,0.95) 0%, 
        rgba(248,250,252,0.98) 100%);
    backdrop-filter: blur(20px);
    border-radius: 20px;
    padding: 2rem;
    margin: 1.5rem 0;
    box-shadow: 
        0 20px 25px -5px rgba(0,0,0,0.1),
        0 10px 10px -5px rgba(0,0,0,0.04),
        0 0 0 1px rgba(255,255,255,0.5);
    position: relative;
    z-index: 1;
    border: 1px solid rgba(255,255,255,0.3);
}
.controls-panel::before {
    content: '';
    position: absolute;
    top: 0;
    left: 0;
    right: 0;
    height: 3px;
    background: linear-gradient(90deg, var(--primary-blue), var(--secondary-purple), var(--accent-cyan));
    border-radius: 20px 20px 0 0;
}

.viewer-border {
    /* initial state: no border, off-screen */
    border: 4px solid transparent;
    opacity: 0;
    transform: translateY(20px);
    transition: opacity 0.5s ease-out, transform 0.5s ease-out, border-color 0.5s ease-out;
}

.info-panel {
    background: linear-gradient(135deg, 
        rgba(255,255,255,0.95) 0%, 
        rgba(240,249,255,0.98) 50%,
        rgba(245,243,255,0.95) 100%);
    backdrop-filter: blur(20px);
    border-radius: 20px;
    padding: 2rem;
    margin: 1.5rem 0;
    box-shadow: 
        0 20px 25px -5px rgba(0,0,0,0.1),
        0 10px 10px -5px rgba(0,0,0,0.04),
        0 0 0 1px rgba(255,255,255,0.5);
    position: relative;
    z-index: 1;
    border: 1px solid rgba(255,255,255,0.3);
}
.info-panel::before {
    content: '';
    position: absolute;
    top: 0;
    left: 0;
    right: 0;
    height: 3px;
    background: linear-gradient(90deg, var(--success-green), var(--accent-cyan), var(--secondary-purple));
    border-radius: 20px 20px 0 0;
}

.protein-title {
    color: var(--primary-blue-dark);
    font-size: 1.5rem;
    font-weight: 700;
    margin-bottom: 1.5rem;
    display: flex;
    align-items: center;
    gap: 0.75rem;
    padding-bottom: 0.5rem;
    border-bottom: 2px solid rgba(37,99,235,0.1);
}
.protein-description {
    color: var(--neutral-700);
    line-height: 1.8;
    font-size: 1rem;
    font-weight: 400;
}
.protein-description strong {
    color: var(--neutral-800);
    font-weight: 600;
}

.temperature-display {
    /* default gradient, overridden after launch */
    background: linear-gradient(135deg, var(--primary-blue), var(--primary-blue-dark));
    color: white;
    padding: 1rem 2rem;
    border-radius: 16px;
    font-weight: 600;
    text-align: center;
    margin: 1rem 0;
    box-shadow: 
        0 10px 15px -3px rgba(37,99,235,0.4),
        0 4px 6px -2px rgba(37,99,235,0.2);
    font-size: 1.1rem;
    letter-spacing: 0.01em;
    position: relative;
    overflow: hidden;
}
.temperature-display::before {
    content: '';
    position: absolute;
    top: 0;
    left: -100%;
    width: 100%;
    height: 100%;
    background: linear-gradient(90deg, transparent, rgba(255,255,255,0.2), transparent);
    animation: shimmer 3s infinite;
}
@keyframes shimmer {
    0% { left: -100%; }
    100% { left: 100%; }
}

.loading-spinner {
    border: 4px solid rgba(37,99,235,0.2);
    border-radius: 50%;
    border-top: 4px solid var(--primary-blue);
    width: 40px;
    height: 40px;
    animation: spin 1s linear infinite;
    margin: 2rem auto;
    position: relative;
}
.loading-spinner::after {
    content: '';
    position: absolute;
    top: 2px;
    left: 2px;
    right: 2px;
    bottom: 2px;
    border-radius: 50%;
    border: 2px solid transparent;
    border-top-color: var(--accent-cyan);
    animation: spin 0.8s linear infinite reverse;
}
@keyframes spin {
    0% { transform: rotate(0deg); }
    100% { transform: rotate(360deg); }
}

.controls-title {
    margin: 0 0 0.5rem 0; 
    color: var(--primary-blue-dark); 
    font-weight: 700;
    font-size: 1.3rem;
    display: flex;
    align-items: center;
    gap: 0.5rem;
}
.controls-description {
    color: var(--neutral-600); 
    margin: 0 0 1.5rem 0; 
    font-size: 1rem;
    line-height: 1.6;
}

.slider-container {
    background: rgba(255,255,255,0.5);
    padding: 1rem 1.5rem;
    border-radius: 12px;
    border: 1px solid rgba(255,255,255,0.6);
    backdrop-filter: blur(10px);
    display: flex;
    flex-direction: column;
    gap: 0.75rem;
    overflow: hidden;
}

.slider-label {
    font-size: 1.2rem;
    font-weight: 700;
    text-align: left;
    margin-bottom: 0.5rem;
}

input[type="range"] {
    height: 24px; /* thicker track */
}
input[type="range"]::-webkit-slider-thumb {
    width: 48px;
    height: 48px;
}
input[type="range"]::-moz-range-thumb {
    width: 48px;
    height: 48px;
}

.button-container {
    /* disable scrollbars */
    overflow: hidden;
    display: flex;
    gap: 0.5rem;
    width: 100%;
}

.enhanced-button {
    background: linear-gradient(135deg, var(--primary-blue), var(--primary-blue-dark)) !important;
    color: white !important;
    border: none !important;
    padding: 14px 28px !important;
    border-radius: 12px !important;
    font-weight: 600 !important;
    font-size: 1rem !important;
    cursor: pointer !important;
    box-shadow: 
        0 10px 15px -3px rgba(37,99,235,0.4),
        0 4px 6px -2px rgba(37,99,235,0.2) !important;
    transition: all 0.3s cubic-bezier(0.4,0,0.2,1) !important;
    position: relative !important;
    overflow: hidden !important;
    letter-spacing: 0.01em !important;
}
.enhanced-button:hover {
    transform: translateY(-2px) !important;
    box-shadow: 
        0 20px 25px -5px rgba(37,99,235,0.4),
        0 10px 10px -5px rgba(37,99,235,0.3) !important;
}
.enhanced-button:active {
    transform: translateY(0px) !important;
}
.enhanced-button::before {
    content: '';
    position: absolute;
    top: 0;
    left: -100%;
    width: 100%;
    height: 100%;
    background: linear-gradient(90deg, transparent, rgba(255,255,255,0.2), transparent);
    transition: left 0.5s;
}
.enhanced-button:hover::before {
    left: 100%;
}

.download-button-enhanced {
    background: linear-gradient(135deg, var(--success-green), #059669) !important;
    color: white !important;
    border: none !important;
    padding: 14px 24px !important;
    border-radius: 12px !important;
    font-weight: 600 !important;
    font-size: 1rem !important;
    cursor: pointer !important;
    box-shadow: 
        0 10px 15px -3px rgba(16,185,129,0.4),
        0 4px 6px -2px rgba(16,185,129,0.2) !important;
    transition: all 0.3s cubic-bezier(0.4,0,0.2,1) !important;
    text-decoration: none !important;
    display: inline-block !important;
    letter-spacing: 0.01em !important;
    height: 50px;
}
.download-button-enhanced:hover {
    transform: translateY(-2px) !important;
    box-shadow: 
        0 20px 25px -5px rgba(16,185,129,0.4),
        0 10px 10px -5px rgba(16,185,129,0.3) !important;
    color: white !important;
    text-decoration: none !important;
}

.experimental-section {
    background: linear-gradient(135deg, 
        rgba(255,255,255,0.95) 0%, 
        rgba(240,249,255,0.98) 50%,
        rgba(245,243,255,0.95) 100%);
    backdrop-filter: blur(20px);
    border-radius: 20px;
    padding: 2rem;
    margin: 2rem 0;
    box-shadow: 
        0 20px 25px -5px rgba(0,0,0,0.1),
        0 10px 10px -5px rgba(0,0,0,0.04),
        0 0 0 1px rgba(255,255,255,0.5);
    position: relative;
    z-index: 1;
    border: 1px solid rgba(255,255,255,0.3);
}
.experimental-section::before {
    content: '';
    position: absolute;
    top: 0;
    left: 0;
    right: 0;
    height: 3px;
    background: linear-gradient(90deg, var(--warning-amber), var(--accent-cyan), var(--secondary-purple));
    border-radius: 20px 20px 0 0;
}
.experimental-title {
    text-align: center; 
    margin: 0 0 2rem 0;
    color: var(--neutral-800);
    font-weight: 700;
    font-size: 1.75rem;
    position: relative;
}
.experimental-title::after {
    content: '';
    position: absolute;
    bottom: -0.5rem;
    left: 50%;
    transform: translateX(-50%);
    width: 60px;
    height: 3px;
    background: linear-gradient(90deg, var(--primary-blue), var(--secondary-purple));
    border-radius: 2px;
}

.chart-container {
    background: rgba(255,255,255,0.8);
    border-radius: 16px;
    padding: 1.5rem;
    margin: 1rem;
    box-shadow: 
        0 10px 15px -3px rgba(0,0,0,0.08),
        0 4px 6px -2px rgba(0,0,0,0.04);
    border: 1px solid rgba(255,255,255,0.6);
    transition: transform 0.3s ease, box-shadow 0.3s ease;
    height: 100%;
    display: flex;
    flex-direction: column;
}
.chart-container:hover {
    transform: translateY(-4px);
    box-shadow: 
        0 20px 25px -5px rgba(0,0,0,0.12),
        0 10px 10px -5px rgba(0,0,0,0.08);
}
.chart-title {
    text-align: center;
    font-weight: 600;
    color: var(--neutral-800);
    margin-bottom: 1rem;
    font-size: 1.1rem;
}
.chart-insight {
    text-align: center; 
    font-size: 0.9rem; 
    color: var(--neutral-600);
    margin-top: 1rem;
    line-height: 1.5;
}

.main-layout {
    padding: 1.5rem;
    max-width: 1400px;
    margin: 0 auto;
}

.viewer-title {
    margin: 0 0 1.5rem 0; 
    color: var(--primary-blue-dark); 
    font-weight: 700; 
    text-align: center;
    font-size: 1.4rem;
    position: relative;
}
.viewer-title::after {
    content: '';
    position: absolute;
    bottom: -0.5rem;
    left: 50%;
    transform: translateX(-50%);
    width: 40px;
    height: 2px;
    background: var(--primary-blue);
    border-radius: 2px;
}

@media (max-width: 768px) {
    .polar-title {
        font-size: 2.2rem;
    }
    .polar-subtitle {
        font-size: 1rem;
    }
    .controls-panel, .info-panel, .viewer-container {
        padding: 1.5rem;
        margin: 0 0;
    }
    .main-layout {
        padding: 1rem;
    }
}
</style>
"""

# Read the PNG file (assumed to live alongside the notebook)
with open("sklab_logo.png", "rb") as f:
    raw = f.read()

# Base64‐encode it
b64 = base64.b64encode(raw).decode("utf-8")

# Build an <img> tag that uses a percentage width
img_tag = (
    f'<img src="data:image/png;base64,{b64}" '
    'style="width:50%; max-width:500px; height:auto; display:block; margin:0 auto;" '
    'alt="SK-LAB Logo" />'
)

# Now insert that into title_html:
title_html = custom_css + f"""
<div class="polar-container">
    {img_tag}
    <p class="polar-subtitle">Advanced Protein Structure Visualization Platform</p>
</div>
"""

title = widgets.HTML(title_html)

# ─────────────────────────────────────────────────────────────────────────────
# 1) Define the "Launch 3D Visualization" button
# ─────────────────────────────────────────────────────────────────────────────
run_button = widgets.Button(
    description="Launch 3D Visualization", 
    button_style='',
    layout=widgets.Layout(height='50px', width='auto', flex='1 1 auto')
)
run_button.add_class('enhanced-button')

# ─────────────────────────────────────────────────────────────────────────────
# 2) Define the slider range & optimum window (no IntText)
# ─────────────────────────────────────────────────────────────────────────────
optimum_min = -32
optimum_max = -28
optimum_center = (optimum_min + optimum_max) // 2  # = -30

slider_label = widgets.HTML("<span class='slider-label'>Adjust Temperature:</span>")

temp_slider = widgets.IntSlider(
    value=optimum_center,
    min=-50,
    max=0,
    step=1,
    description="",  # hide default description
    style={'description_width': '0px'},
    layout=widgets.Layout(width='100%')  # full width of its container
)

# ─────────────────────────────────────────────────────────────────────────────
# 3) Define the download button (fixed size, same height as run_button)
# ─────────────────────────────────────────────────────────────────────────────
download_html = """
<div style="display:inline-block; visibility:hidden;" id="download‐wrapper">
    <a download='' href='' target='_blank' class="download-button-enhanced">
        <i class="fas fa-download" aria-hidden="true"></i>
    </a>
</div>
"""
download_button = widgets.HTML(download_html)
download_button.layout = widgets.Layout(
    width='120px',
    height='50px',
    visibility='hidden'
)

# ─────────────────────────────────────────────────────────────────────────────
# 4) Slider container: holds only label + slider
# ─────────────────────────────────────────────────────────────────────────────
slider_container_box = widgets.VBox(
    [slider_label, temp_slider],
    layout=widgets.Layout(width='100%')
)
slider_container_box.add_class("slider-container")

# ─────────────────────────────────────────────────────────────────────────────
# 5) Configure run_button and download_button to share container
# ─────────────────────────────────────────────────────────────────────────────
button_container = widgets.HBox(
    [run_button, download_button],
    layout=widgets.Layout(
        width='100%',      # ensure HBox spans full panel width
        overflow='hidden', # disable scrolling
        align_items='center'
    )
)

# ─────────────────────────────────────────────────────────────────────────────
# 6) Build control panel header/title & description
# ─────────────────────────────────────────────────────────────────────────────
controls_title = widgets.HTML(
    "<h3 class='controls-title'>🎛️ Visualization Controls</h3>"
)
controls_description = widgets.HTML(
    "<p class='controls-description'>"
    "Adjust the temperature to explore different protein conformations under Arctic conditions. "
    "Each temperature setting reveals unique structural adaptations that enable polar bear survival "
    "in extreme environments."
    "</p>"
)

# ─────────────────────────────────────────────────────────────────────────────
# 7) Assemble the entire control panel
#    Order: [ title, description, slider, buttons ]
# ─────────────────────────────────────────────────────────────────────────────
controls_panel_box = widgets.VBox(
    [
        controls_title,
        controls_description,
        slider_container_box,
        button_container
    ],
    layout=widgets.Layout(
        padding="1rem",
        margin="1rem 0"
    )
)
controls_panel_box.add_class("controls-panel")
controls_info = controls_panel_box

# ─────────────────────────────────────────────────────────────────────────────
# Protein Info Panel (unchanged)
# ─────────────────────────────────────────────────────────────────────────────
protein_info_html = """
<div class="info-panel">
    <div class="protein-title">
        Aquaporin-3 (AQP3) - Arctic Adaptation Protein
    </div>
    <div class="protein-description">
        <strong>Functional Overview:</strong> Aquaporin-3 is a sophisticated membrane channel protein that facilitates the selective transport of water and small solutes including glycerol across cellular membranes.
        <br><br>
        <strong>Arctic Specialization:</strong> In the extreme Arctic environment inhabited by polar bears, AQP3 plays a crucial role in cellular survival mechanisms. The protein helps maintain optimal cellular hydration and membrane fluidity at subzero temperatures by precisely regulating intracellular glycerol concentrations.
        <br><br>
        <strong>Cryoprotective Function:</strong> Glycerol acts as a natural antifreeze compound (cryoprotectant), and AQP3's expression in polar bear skin and hair follicles contributes to remarkable ice resistance by minimizing ice crystal nucleation and preserving cellular structural integrity under severe freezing stress.
    </div>
</div>
"""
protein_info = widgets.HTML(protein_info_html)

# ─────────────────────────────────────────────────────────────────────────────
# 8) Function to compute slider‐handle color based on proximity to optimum
# ─────────────────────────────────────────────────────────────────────────────
def compute_slider_color(temp: int) -> str:
    # If inside optimum band, return pure green
    if optimum_min <= temp <= optimum_max:
        return "#00ff00"
    # Otherwise, compute absolute distance to center
    d_abs = abs(temp - optimum_center)
    # Maximum possible distance from center is 30° (from -30 to 0)
    max_ref = abs(0 - optimum_center)  # = 30
    norm = min(d_abs / max_ref, 1.0)
    # Interpolate R, G between red and green
    r = int(norm * 255)
    g = int((1.0 - norm) * 255)
    b = 0
    return f"#{r:02x}{g:02x}{b:02x}"

# ─────────────────────────────────────────────────────────────────────────────
# 9) Experimental Graphics Section will be displayed after a delay
# ─────────────────────────────────────────────────────────────────────────────
def create_graphics_section():
    chart_configs = [
        {
            "title": "Lipid Class Abundance",
            "endpoint": "/api/lipid/lipid_chart",
            "insight": "Lipid class distribution in polar bear samples."
        },
        {
            "title": "Fatty Acid Abundance (Bar)",
            "endpoint": "/api/abundance/fa_bar_chart",
            "insight": "Fatty acids vary with environmental exposure."
        },
        {
            "title": "Fatty Acid Composition (Pie)",
            "endpoint": "/api/abundance/fa_pie_chart",
            "insight": "Most abundant fatty acids shown proportionally."
        },
        {
            "title": "NMR Quantification (Stacked Bar)",
            "endpoint": "/api/nmr-quant/stacked-bar",
            "insight": "Relative lipid concentrations per sample."
        },
        {
            "title": "NMR Quantification (Heatmap)",
            "endpoint": "/api/nmr-quant/heatmap",
            "insight": "Heatmap reveals concentration trends."
        }
    ]
    base_url = "http://127.0.0.1:5000"
    chart_widgets = []

    for chart in chart_configs:
        url = f"{base_url}{chart['endpoint']}"
        try:
            resp = requests.get(url)
            if resp.status_code == 200:
                # Encode PNG bytes as base64 so we can inline it in one HTML widget
                b64_png = base64.b64encode(resp.content).decode('utf-8')

                html_snippet = f"""
                <div class="chart-container">
                    <h4 class="chart-title">{chart['title']}</h4>
                    <img src="data:image/png;base64,{b64_png}" style="max-width:100%; height:auto;" />
                    <p class="chart-insight">{chart['insight']}</p>
                </div>
                """
                chart_widgets.append(
                    widgets.HTML(
                        html_snippet,
                        layout=widgets.Layout(
                            flex="1 1 300px",    # grow/shrink, basis 300px
                            min_width="280px",   # don’t shrink below ~280px
                            margin="0.5rem"      # small gap around each container
                        )
                    )
                )
        except Exception:
            chart_widgets.append(
                widgets.HTML(
                    f"<div class='chart-container'>"
                    f"  <p style='color:red; text-align:center;'>Failed: {chart['title']}</p>"
                    f"</div>",
                    layout=widgets.Layout(
                        flex="1 1 300px",
                        min_width="280px",
                        margin="0.5rem"
                    )
                )
            )

    charts_box = widgets.Box(
        children=chart_widgets,
        layout=widgets.Layout(
            display="flex",
            flex_flow="row wrap",      # allow wrapping to next line automatically
            justify_content="space-around",
            align_items="stretch",
            width="100%",
            overflow="hidden"
        )
    )

    return widgets.VBox([
        widgets.HTML("<h2 class='experimental-title'>Experimental Insights</h2>"),
        charts_box
    ])

# We'll put the actual insights inside an Output widget so we can insert a delay
graphics_output = widgets.Output()

# ─────────────────────────────────────────────────────────────────────────────
# 10) Output area for 3D viewer and temperature status
# ─────────────────────────────────────────────────────────────────────────────
viewer_output = widgets.Output()
temp_status = widgets.HTML()

def update_temp_status(temp, post_launch_color=None):
    # If post_launch_color is provided, override the background
    bgcolor = post_launch_color or "linear-gradient(135deg, var(--primary-blue), var(--primary-blue-dark))"
    status_html = f"""
        <div class="temperature-display" style="background: {bgcolor};">
            <strong style="font-size:1.2rem;">Current Arctic Temperature: {temp}°C</strong>
            | Arctic Condition: {'Severe' if temp <= -40 else 'Extreme' if temp <= -30 else 'Moderate' if temp <= -20 else 'Mild'}
        </div>
    """
    temp_status.value = status_html

# ─────────────────────────────────────────────────────────────────────────────
# 11) Bind Run button click → show 3D viewer + border, then after 5s show insights
# ─────────────────────────────────────────────────────────────────────────────
def on_run_click(b):
    temp = temp_slider.value
    slider_color = compute_slider_color(temp)

    with viewer_output:
        clear_output()
        # Show loading spinner for viewer
        print("🔄 Initializing 3D molecular visualization...")
        loading_html_viewer = (
            '<div class="loading-spinner"></div>'
            '<p style="text-align: center; color: #6b7280; '
            'font-weight: 500; margin-top: 1rem;">'
            'Loading protein structure...</p>'
        )
        display(HTML(loading_html_viewer))

        # Read CIF content and display viewer
        cif_path = get_model_path_from_temperature(temp)
        with open(cif_path, "r") as f:
            cif_str = f.read()

        clear_output()
        # Render viewer (no border)
        display(HTML(
            '<h3 style="margin: 0 0 1rem 0; '
            'color: #1e40af; font-weight: 600; '
            'text-align: center;">'
            '3D Molecular Structure Viewer'
            '</h3>'
        ))
        view = py3Dmol.view(width='100%', height=500)
        view.addModel(cif_str, "cif")
        view.setStyle({"cartoon": {"color": "spectrum"}})
        view.setBackgroundColor("#f8fafc")
        view.zoomTo()
        view.show()

        # Update the temperature banner with the current slider value
        update_temp_status(temp, post_launch_color=slider_color)

    # ─────────────────────────────────────────────────────────────────────────
    # Now un-hide & re-wire the download button to point to the current CIF file
    # ─────────────────────────────────────────────────────────────────────────
    # 1) Get just the filename (basename) from the full cif_path:
    file_name = os.path.basename(cif_path)

    # Ensure CIF file is copied to notebook's directory for download
    notebook_dir = os.getcwd()
    target_cif_path = os.path.join(notebook_dir, file_name)

    if not os.path.isfile(target_cif_path):
        from shutil import copyfile
        copyfile(cif_path, target_cif_path)

    # 2) Construct new HTML for download, inserting the correct href and download name:
    new_download_html = f"""
    <div style="display:inline-block; visibility:visible;" id="download‐wrapper">
        <a download="{file_name}" href="{file_name}" 
        target='_blank' class="download-button-enhanced">
            <i class="fas fa-download" aria-hidden="true"></i>
        </a>
    </div>
    """

    # 3) Overwrite the existing download_button.value with the new HTML:
    download_button.value = new_download_html

    # 4) Also make sure the widget itself is visible:
    download_button.layout.visibility = 'visible'

    # Immediately display the insights (no delay)
    with graphics_output:
        clear_output()
        display(create_graphics_section())

# ─────────────────────────────────────────────────────────────────────────────
# 12) Bind slider change → update only the slider handle color (no status update)
# ─────────────────────────────────────────────────────────────────────────────
def on_temp_change(change):
    new_temp = change['new']
    # Only update the slider handle color
    color_hex = compute_slider_color(new_temp)
    temp_slider.style.handle_color = color_hex

temp_slider.observe(on_temp_change, names='value')

# Initial handle color (pre-launch)
initial_color = compute_slider_color(temp_slider.value)
temp_slider.style.handle_color = initial_color
# Initialize temp_status with default gradient (but do not update number yet)
update_temp_status(temp_slider.value)

# Bind Run button
run_button.on_click(on_run_click)

# ─────────────────────────────────────────────────────────────────────────────
# 13) Assemble the full layout: title → viewer → status → controls → protein info → graphics
# ─────────────────────────────────────────────────────────────────────────────
layout = widgets.VBox([
    title,
    viewer_output,
    temp_status,
    controls_info,
    protein_info,
    graphics_output   # initially empty; spinner & insights appear after launch
], layout=widgets.Layout(padding='1rem'))

display(layout)